#### LZ码

将变长的输入符号串映射成定长或长度可预测的码字，按照几乎相等的出现概率安排输入符号串，从而使频繁出现符号的串将比不常出现符号的串包含更多的符号。

压缩算法是自适应的，只需扫描一次数据，无需有关数据统计量的先验信息，运算时间正比于消息的长度。

#### LZW算法

建立转换表，若由某个字符串ω和某个单字符K所组成的字符串ωK在表中，则ω也在表中。

流程

初始化：将所有单字符串放入串表

    读第一个输入字符 → 前缀串ω

Step:读下一个输入字符串K

    if 没有K(输入已穷尽):

        码字(ω) → 输出；结束

    if ωK已存在于串表中:

        ωK → ω; Repeat Step

    else ωK不在串表中:

        码字(ω) → 输出

        ωK → 串表;

        K → ω; Repeat Step

In [1]:
# 例，对三字母字符串做LZW编码，结果为4位码字
str = "ababcbababaaaaaaa";
maplist = ["a","b","c"];
ω = str[1];
cstr = "";
for i in str[2:end]
    K = i;
    ωK = string(ω, K);
    if ωK in maplist
        ω = ωK;
    else
        cstr = string(cstr, string(findfirst(maplist .== "$ω"),base=16,pad=1));
        push!(maplist, ωK);
        ω = K;
    end
end
cstr = string(cstr, string(findfirst(maplist .== "$ω"),base=16,pad=1));
@show maplist;
@show cstr;

maplist = ["a", "b", "c", "ab", "ba", "abc", "cb", "bab", "baba", "aa", "aaa", "aaaa"]
cstr = "1243581ab1"


In [5]:
length(str) > length(cstr)

true

In [4]:
# 另一种形式的字串表，以前缀标识符加扩充字符表示新字串，优点是表中新增的每一项长度相等
str = "ababcbababaaaaaaa";
maplist = ["a","b","c"];
ω = str[1];
cstr = "";
for i in str[2:end]
    K = i;
    idx = string(findfirst(maplist .== "$ω"),base=16,pad=1);
    ωK = "$idx$K";
    if ωK in maplist
        ω = ωK;
    else
        cstr = string(cstr, idx);
        push!(maplist, ωK);
        ω = K;
    end
end
cstr = string(cstr, string(findfirst(maplist .== "$ω"),base=16,pad=1));
@show maplist;
@show cstr;

maplist = ["a", "b", "c", "1b", "2a", "4c", "3b", "5b", "8a", "1a", "aa", "ba"]
cstr = "1243581ab1"


LZW解码算法，核心在于还原编码时用的字典，参考[link](https://blog.csdn.net/hanzhen7541/article/details/91141112)

流程

初始化：将所有已知码字放入串表

    读第一个输入码字 → p；输出字符串(p)

Step:c → p

    读下一个输入码字c

    if 没有c(输入已穷尽):

        字符串(c) → 输出；结束

    if c已存在于串表中:

        输出字符串(c)

        字符串(p) → P; 字符串(c) → C

        字典中加入P+C[1]; Repeat Step

    else c不在串表中:

        字符串(p) → P; 字符串(p) → C

        字典中加入P+C[1]; 输出P+C[1]; Repeat Step

In [6]:
rmaplist = ["a","b","c"];
c = parse(Int,cstr[1],base=16);
str2 = rmaplist[c];
N = length(cstr);
for i in 2:N
    p = c;
    c = parse(Int,cstr[i],base=16);
    if c <= length(rmaplist)
        P = rmaplist[p]; C = rmaplist[c];
        str2 = "$str2$C";
        push!(rmaplist,string(P,C[1]));
    else
        P = rmaplist[p]; C = P[1];
        str2 = "$str2$P$C";
        push!(rmaplist,string(P,C));
    end
end
@show rmaplist;
@show str2;

rmaplist = ["a", "b", "c", "ab", "ba", "abc", "cb", "bab", "baba", "aa", "aaa", "aaaa"]
str2 = "ababcbababaaaaaaa"


In [14]:
isequal(str,str2)

true

字典的空间有限，当数据量很大时如果字典被填满，不考虑算法改进的话有两种选择

1.扩大字典容量与码字长度

2.初始化字典，从该位置继续编码

In [11]:
str = "ababcbababaaaaaaa";
maplist = ["a","b","c"];
codelen = 3;
ω = string(str[1]);
carray = [];
for i in str[2:end]
    K = string(i);
    ωK = "$ω$K";
    if ωK in maplist
        ω = ωK;
    else
        push!(carray, findfirst(maplist .== "$ω"));
         # 如果字典已满则重置字典
        if length(maplist) == 2^codelen
            println(maplist);
            maplist = ["a","b","c"];
        else
            push!(maplist, ωK);
        end
        ω = K;
    end
end
push!(carray, findfirst(maplist .== "$ω"));
println(maplist," ",length(maplist));
println(carray," ",length(carray));

["a", "b", "c", "ab", "ba", "abc", "cb", "bab"]
["a", "b", "c", "aa", "aaa", "aaaa"] 6
Any[1, 2, 4, 3, 5, 8, 1, 4, 5, 1] 10


In [20]:
rmaplist = ["a","b","c"];
c = carray[1];
array2 = [rmaplist[c]];
N = length(carray);
i = 2;
while i <= N
    p = c;
    c = carray[i];
    if c <= length(rmaplist)
        P = rmaplist[p]; C = rmaplist[c];
        push!(array2, "$C");
        if length(rmaplist) == 2^codelen
            println(rmaplist);
            rmaplist = ["a","b","c"];
        else 
            push!(rmaplist,string(P,C[1]));
        end
    else
        P = rmaplist[p]; C = P[1];
        push!(array2, "$P$C");
        if length(rmaplist) == 2^codelen
            println(rmaplist);
            rmaplist = ["a","b","c"];
        else
            push!(rmaplist,"$P$C");
        end
    end
    i += 1;
end
println(rmaplist);
println(array2);

["a", "b", "c", "ab", "ba", "abc", "cb", "bab"]
["a", "b", "c", "aa", "aaa", "aaaa"]
["a", "b", "ab", "c", "ba", "bab", "a", "aa", "aaa", "a"]
